### Extract SGP & instances from files

In [1]:
from SGP import SGP_

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/nlplab/jocelyn/stanfordnlp_resources/zh_gsd_models/zh_gsd_tokenizer.pt', 'pretokenized': True, 'lang': 'zh', 'shorthand': 'zh_gsd', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/nlplab/jocelyn/stanfordnlp_resources/zh_gsd_models/zh_gsd_tagger.pt', 'pretrain_path': '/home/nlplab/jocelyn/stanfordnlp_resources/zh_gsd_models/zh_gsd.pretrain.pt', 'lang': 'zh', 'shorthand': 'zh_gsd', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/nlplab/jocelyn/stanfordnlp_resources/zh_gsd_models/zh_gsd_lemmatizer.pt', 'lang': 'zh', 'shorthand': 'zh_gsd', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/nlplab/jocelyn/stanfordnlp_resources/zh_gsd_models/zh_gsd_parser.pt', 'pretrai

In [4]:
file_path = 'data/cambridge_align.txt'
align_file_path = 'data/cambridge.align'
ch_pos_file_path = 'data/cambridge_ch_pos.txt'

SGP_.process_file(file_path, align_file_path, ch_pos_file_path, 40)

data/cambridge_align.txt data/cambridge.align data/cambridge_ch_pos.txt


In [6]:
SGP_.verb_dict['buy']

defaultdict(<function SGP.SGP.__init__.<locals>.<lambda>.<locals>.<lambda>()>,
            {'V n': defaultdict(list,
                         {'V n': [('[bought] a car',
                            '[買] 輛 汽車',
                            'I have [bought] a car .',
                            '我 [買] 了 輛 汽車 。')]})})

#### 統計所有例句得到 frequency & 並過濾掉 pattern freq <= 2% 的 pattern & 用 GDEX score 排序例句

In [8]:
from collections import defaultdict, Counter
from GDEX import GDEX

def get_count(verb_dict):
    verb_pat_count = defaultdict(lambda: defaultdict(Counter)) # verb_pat_count[en_verb][en_pat][ch_pat] = count
    verb_count = Counter() # verb_count[verb] = count
    ch_lens = []; en_lens = []
    
    for en_verb in verb_dict:
        for en_pat in verb_dict[en_verb]:
            for ch_pat, examples in verb_dict[en_verb][en_pat].items():
                verb_pat_count[en_verb][en_pat][ch_pat] = len(examples)
                for example in examples:
                    ch_lens += [len(example[3].split())]
                    en_lens += [len(example[2].split())]
            verb_count[en_verb] += sum(verb_pat_count[en_verb][en_pat].values())
    
    ch_len_avg = int(sum(ch_lens)/len(ch_lens))
    en_len_avg = int(sum(en_lens)/len(en_lens))
    
    return ch_len_avg, en_len_avg, verb_pat_count, verb_count

In [9]:
def new_file(verb_dict):
    verb_dict_new = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list)))) # 加上 freq 的版本
    
    # Step1: 先從原始的 SGP result 計算出例句＆動詞的 count ＆ 例句平均長度
    ch_len_avg, en_len_avg, verb_pat_count, verb_count = get_count(verb_dict)
    
    # Step2: 把原始的 SGP result 加上 frequency
    for en_verb in verb_dict:
        for en_pat in verb_dict[en_verb]:
            en_freq = 100*sum(verb_pat_count[en_verb][en_pat].values())/verb_count[en_verb]
            if en_freq <= 2:
                continue
            for ch_pat, examples in verb_dict[en_verb][en_pat].items():
                ch_freq = 100*len(examples)/sum(verb_pat_count[en_verb][en_pat].values())
                if ch_freq <= 2:
                    continue
                verb_dict_new[en_verb][en_pat][ch_pat]['examples'] = GDEX.rank_examples(examples, en_len_avg, ch_len_avg)
                verb_dict_new[en_verb][en_pat][ch_pat]['freq'] = int(ch_freq) # round(ch_freq, 2)
            verb_dict_new[en_verb][en_pat]['freq'][''] = int(en_freq) # round(en_freq, 2)
            
    return verb_dict_new

In [10]:
verb_dict_new = new_file(SGP_.verb_dict)

In [12]:
verb_dict_new['buy']

defaultdict(<function __main__.new_file.<locals>.<lambda>.<locals>.<lambda>()>,
            {'V n': defaultdict(<function __main__.new_file.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'V n': defaultdict(list,
                                      {'examples': [('[bought] a car',
                                         '[買] 輛 汽車',
                                         'I have [bought] a car .',
                                         '我 [買] 了 輛 汽車 。')],
                                       'freq': 100}),
                          'freq': defaultdict(list, {'': 100})})})

#### Write to json file

In [13]:
SGP_.write_to_file()

In [15]:
import json

with open('SGP_result_2.json', 'w') as outf:
    json.dump(verb_dict_new, outf, ensure_ascii=False)